In [23]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# Importing CSV Files

In [187]:
df_BTC = pd.read_csv("BTC_Historical_11-03.csv", index_col=0)
df_DOGE = pd.read_csv("DOGE_Historical_11-03.csv", index_col=0)
df_XRP = pd.read_csv("XRP_Historical_11-03.csv", index_col=0)
df_ETH = pd.read_csv("ETH_Historical_11-03.csv", index_col=0)
df_UNI = pd.read_csv("UNI_Historical_11-03.csv", index_col=0)
df_BTC_copy = pd.read_csv("BTC_Historical_11-03.csv", index_col=0)


# LINREQ

In [32]:
# BTC

In [132]:
degree = 2  # Degree of polynomial features
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_BTC[['Closing Price (USD)']]
y = df_BTC['y']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
last_available_data = df_BTC.iloc[-1]
future_X = pd.DataFrame({'Closing Price (USD)': [last_available_data['Closing Price (USD)']]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 69853.49647791797


In [134]:
degree = 2  # Degree of polynomial features
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_BTC[['Closing Price (USD)']]
y = df_BTC['Lowest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
last_available_data = df_BTC.iloc[-1]
future_X = pd.DataFrame({'Closing Price (USD)': [last_available_data['Closing Price (USD)']]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted lowest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted lowest prices for future dates:
2024-03-12 00:00:00: 66348.70923459053


In [144]:
btc_high_pred = 69853.49647791797
btc_low_pred = 66348.70923459053

In [170]:
df_BTC_pred = df_BTC[["y", "Lowest Price (USD)"]]
new_date = pd.to_datetime('2024-03-12')
pred_value = [btc_high_pred, btc_low_pred]
df_BTC_pred.loc[new_date] = pred_value

/var/folders/sz/x51nf6p57096mncqbrxctq940000gn/T/ipykernel_10389/2375387423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BTC_pred.loc[new_date] = pred_value


In [188]:
# BTC copy

In [235]:
degree = 5

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_BTC_copy[['Closing Price (USD)']]
y = df_BTC['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [68526]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 71999.8493557355


In [236]:
degree = 5

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_BTC_copy[['Closing Price (USD)']]
y = df_BTC_copy['Lowest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [68526]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted lowest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted lowest prices for future dates:
2024-03-12 00:00:00: 68756.58238189445


In [205]:
btc_high_pred = 71999.8493557355
btc_low_pred = 68756.58238189445

In [33]:
# DOGE

In [240]:
degree = 5

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_DOGE[['Closing Price (USD)']]
y = df_DOGE['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [0.17]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 0.179185988270293


In [239]:
degree = 5

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_DOGE[['Closing Price (USD)']]
y = df_DOGE['Lowest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [0.17]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted lowest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted lowest prices for future dates:
2024-03-12 00:00:00: 0.16270525360652066


In [193]:
DOGE_high_pred = 0.179185988270293
DOGE_low_pred = 0.16270525360652066

In [38]:
# Etherum

In [237]:
degree = 5

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_ETH[['Closing Price (USD)']]
y = df_ETH['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [3817.03]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 3923.0733362897136


In [238]:
degree = 5

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_ETH[['Closing Price (USD)']]
y = df_ETH['Lowest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [3817.03]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted lowest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted lowest prices for future dates:
2024-03-12 00:00:00: 3678.9589765217825


In [196]:
ETH_high_pred = 3923.0733362897136
ETH_low_pred = 3678.9589765217825

In [41]:
# XRP

In [241]:
degree = 5
# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_XRP[['Closing Price (USD)']]
y = df_XRP['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [0.6]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 0.6171763695870979


In [242]:
degree = 5

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_XRP[['Closing Price (USD)']]
y = df_XRP['Lowest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [0.6]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted lowest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted lowest prices for future dates:
2024-03-12 00:00:00: 0.580943608931109


In [199]:
XRP_high_pred = 0.6171763695870979
XRP_low_pred = 0.580943608931109

In [46]:
# UNI

In [243]:
degree = 5


# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_UNI[['Closing Price (USD)']]
y = df_UNI['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

future_dates = pd.date_range(start='2024-03-12', periods=7, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [14.13]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 14.94103422104355


In [244]:
degree = 5


# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_UNI[['Closing Price (USD)']]
y = df_UNI['Lowest Price (USD)']

# Fit the model
polyreg.fit(X, y)

future_dates = pd.date_range(start='2024-03-12', periods=7, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [14.13]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted lowest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted lowest prices for future dates:
2024-03-12 00:00:00: 13.4086553139549


In [151]:
UNI_high_pred = 14.94
UNI_low_pred = 13.41

# Adding the predicted results to the dataframe

In [185]:
def add_new_row(df, highest_price, lowest_price):
    # Make a copy of the original DataFrame with only desired columns
    new_df = df[['Highest Price (USD)', 'Lowest Price (USD)']].copy()
    
    # New data to add
    new_date = pd.to_datetime('2024-03-12')
    new_row = {'Highest Price (USD)': highest_price, 'Lowest Price (USD)': lowest_price}

    # Append the new row
    new_df.loc[new_date.strftime('%Y-%m-%d')] = new_row

    return new_df

In [213]:
df_BTC_pred = add_new_row(df_BTC_copy, btc_high_pred, btc_low_pred)
df_ETH_pred = add_new_row(df_ETH, ETH_high_pred, ETH_low_pred)
df_DOGE_pred = add_new_row(df_DOGE, DOGE_high_pred, DOGE_low_pred)
df_XRP_pred = add_new_row(df_XRP, XRP_high_pred, XRP_low_pred)
df_UNI_pred = add_new_row(df_UNI, UNI_high_pred, UNI_low_pred)

In [247]:
filter_pred = df_UNI_pred[df_UNI_pred.index == '2024-03-12']
filter_pred

,Highest Price (USD),Lowest Price (USD)
2024-03-12,14.94,13.41


# Exporting these dataframes

In [214]:
def exporting_dataframe(df, filename):
    df.to_csv(f'{filename}.csv', index=True)

In [216]:
exporting_dataframe(df_BTC_pred, 'BTC_With_Prediction')
exporting_dataframe(df_ETH_pred, 'ETH_With_Prediction')
exporting_dataframe(df_DOGE_pred, 'DOGE_With_Prediction')
exporting_dataframe(df_XRP_pred, 'XRP_With_Prediction')
exporting_dataframe(df_UNI_pred, 'UNI_With_Prediction')